In [ ]:
from   embed_and_verify import *
from   config import *
from   prune_and_fine_tune_utils import *
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import networkx as nx
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import pandas as pd
import pickle
import random
import requests
from matplotlib import patches, transforms

from   scipy import stats
from   scipy.stats import norm, anderson, kstest, zscore, shapiro

import torch
from   torch.nn import ModuleList
import torch.nn.utils.prune as prune
import torch.nn.functional as F
import torch.optim as optim

from   torch_geometric.datasets import Reddit2
from   torch_geometric.nn import GATConv, GCNConv, GraphConv, SAGEConv, GINConv, global_mean_pool
from   torch_geometric.transforms import RandomLinkSplit
from   torch_geometric.data import Data
from   torch_geometric.loader import DataLoader

import zipfile

from   pcgrad.pcgrad import PCGrad 

mpl.rcParams['figure.dpi']=250

root_folder = '<path_to_repo_dir'
training_folder = os.path.join(root_folder,'training_results')

# Results Printouts

### Classification Only (No Watermarking)

In [ ]:
## Accuracy rates without watermarking

# Examples of results you may want to display:
photo_sage_results_clf_only =  training_folder + '/photo/clf_only_archSAGE_elu_nLayers3_hDim256_drop0.1_skipTrue/clf_only_nodeDropP0.3_nodeMixUp1_nodeFeatMask0.3_edgeDrop0.3_lr0.0002_epochs300'
photo_gcn_results_clf_only =  training_folder + '/photo/clf_only_archGCN_elu_nLayers3_hDim256_drop0.1_skipTrue/clf_only_nodeDropP0.3_nodeMixUp1_nodeFeatMask0.3_edgeDrop0.3_lr0.0002_epochs300'
photo_sgc_results_clf_only =  training_folder + '/photo/clf_only_archSGC_elu_nLayers3_hDim256_drop0.1_skipTrue/clf_only_nodeDropP0.3_nodeMixUp1_nodeFeatMask0.3_edgeDrop0.3_lr0.0002_epochs300'
CS_sage_results_clf_only = training_folder + '/CS/clf_only_archSAGE_elu_nLayers3_hDim256_drop0.1_skipTrue/clf_only_nodeDropP0.1_nodeMixUp5_edgeDrop0.1_lr0.001_epochs90'
CS_gcn_results_clf_only = training_folder + '/CS/clf_only_archGCN_elu_nLayers3_hDim256_drop0.1_skipTrue/clf_only_nodeDropP0.1_nodeMixUp5_edgeDrop0.1_lr0.001_epochs90'
CS_sgc_results_clf_only = training_folder + '/CS/clf_only_archSGC_elu_nLayers3_hDim256_drop0.1_skipTrue/clf_only_nodeDropP0.1_nodeMixUp5_edgeDrop0.1_lr0.001_epochs90'
pubmed_sage_results_clf_only = training_folder + '/PubMed/clf_only_archSAGE_elu_nLayers3_hDim512_drop0.95_skipTrue/clf_only_nodeDropP0.1_nodeMixUp0.6_edgeDrop0.1_lr0.005_epochs200'
pubmed_gcn_results_clf_only = training_folder + '/PubMed/clf_only_archGCN_elu_nLayers3_hDim512_drop0.8_skipTrue/clf_only_nodeDropP0.1_nodeMixUp0.6_edgeDrop0.1_lr0.005_epochs200'
pubmed_sgc_results_clf_only = training_folder + '/PubMed/clf_only_archSGC_elu_nLayers3_hDim512_drop0.8_skipTrue/clf_only_nodeDropP0.1_nodeMixUp0.6_edgeDrop0.1_lr0.005_epochs200'

results_folders = [photo_sage_results_clf_only, photo_gcn_results_clf_only, photo_sgc_results_clf_only, 
                   CS_sage_results_clf_only, CS_gcn_results_clf_only, CS_sgc_results_clf_only,
                   pubmed_sage_results_clf_only,pubmed_gcn_results_clf_only,pubmed_sgc_results_clf_only]

result_names = ['Photo SAGE', 'Photo GCN', 'Photo SGC', 
                'CS SAGE', 'CS GCN','CS SGC', 
                'PubMed SAGE', 'PubMed GCN', 'PubMed SGC']


print("Accuracy without watermarking")
for results_folder,results_name in zip(results_folders,result_names):
    # if 'SAGE' in results_name:
    train_accs = []
    val_accs = []
    test_accs = []
    seeds = [f for f in os.listdir(results_folder) if 'seed' in f and f!='seed0']
    for s in seeds:
        file = results_folder + '/' + s + '/results_clf_only.txt'
        with open(file,'r') as f:
            lines = f.readlines()
        f.close()
        train_acc = float(lines[1].split("(trn/val/test)= ")[1].split('/')[0])
        val_acc = float(lines[1].split("(trn/val/test)= ")[1].split('/')[1])
        test_acc = float(lines[1].split("(trn/val/test)= ")[1].split('/')[2])
        train_accs.append(train_acc)
        val_accs.append(val_acc)
        test_accs.append(test_acc)
    print(f'{f"{results_name}:".ljust(11)} average train/val/test acc = {np.mean(train_accs):.3f}/{np.mean(val_accs):.3f}/{np.mean(test_accs):.3f}')

### Random Backdoor Watermark (Not our method)

In [ ]:
# WATERMARK RANDOM BACKDOOR
alpha=0.4

# Examples of results you may want to display:
photo_sage_random_backdoor  = training_folder + f'/photo/watermark_random_backdoor_archSAGE_elu_nLayers3_hDim256_drop0.1_skipTruepEdge0.05_pOnes0.5_dataProportion0.05_alpha{alpha}/watermark_random_backdoor_nodeDropP0.3_nodeMixUp1_nodeFeatMask0.3_edgeDrop0.3_lr0.0002_epochs300_pcgrad'
photo_gcn_random_backdoor  = training_folder + f'/photo/watermark_random_backdoor_archGCN_elu_nLayers3_hDim256_drop0.1_skipTruepEdge0.05_pOnes0.5_dataProportion0.05_alpha{alpha}/watermark_random_backdoor_nodeDropP0.3_nodeMixUp1_nodeFeatMask0.3_edgeDrop0.3_lr0.0002_epochs300_pcgrad'
photo_sgc_random_backdoor  = training_folder + f'/photo/watermark_random_backdoor_archSGC_elu_nLayers3_hDim256_drop0.1_skipTruepEdge0.05_pOnes0.5_dataProportion0.05_alpha{alpha}/watermark_random_backdoor_nodeDropP0.3_nodeMixUp1_nodeFeatMask0.3_edgeDrop0.3_lr0.0002_epochs300_pcgrad'
CS_sage_random_backdoor            = training_folder + f'/CS/watermark_random_backdoor_archSAGE_elu_nLayers3_hDim256_drop0.1_skipTruepEdge0.05_pOnes0.5_dataProportion0.05_alpha{alpha}/watermark_random_backdoor_nodeDropP0.1_nodeMixUp5_edgeDrop0.1_lr0.001_epochs90_pcgrad'
CS_gcn_random_backdoor            = training_folder + f'/CS/watermark_random_backdoor_archGCN_elu_nLayers3_hDim256_drop0.1_skipTruepEdge0.05_pOnes0.5_dataProportion0.05_alpha{alpha}/watermark_random_backdoor_nodeDropP0.1_nodeMixUp5_edgeDrop0.1_lr0.001_epochs90_pcgrad'
CS_sgc_random_backdoor            = training_folder + f'/CS/watermark_random_backdoor_archSGC_elu_nLayers3_hDim256_drop0.1_skipTruepEdge0.05_pOnes0.5_dataProportion0.05_alpha{alpha}/watermark_random_backdoor_nodeDropP0.1_nodeMixUp5_edgeDrop0.1_lr0.001_epochs90_pcgrad'
pubmed_sage_random_backdoor        = training_folder + f'/PubMed/watermark_random_backdoor_archSAGE_elu_nLayers3_hDim512_drop0.95_skipTruepEdge0.05_pOnes0.5_dataProportion0.05_alpha{alpha}/watermark_random_backdoor_nodeDropP0.1_nodeMixUp0.6_edgeDrop0.1_lr0.005_epochs200_pcgrad'
pubmed_gcn_random_backdoor        = training_folder + f'/PubMed/watermark_random_backdoor_archGCN_elu_nLayers3_hDim512_drop0.95_skipTruepEdge0.05_pOnes0.5_dataProportion0.05_alpha{alpha}/watermark_random_backdoor_nodeDropP0.1_nodeMixUp0.6_edgeDrop0.1_lr0.005_epochs200_pcgrad'
pubmed_sgc_random_backdoor        = training_folder + f'/PubMed/watermark_random_backdoor_archSGC_elu_nLayers3_hDim512_drop0.95_skipTruepEdge0.05_pOnes0.5_dataProportion0.05_alpha{alpha}/watermark_random_backdoor_nodeDropP0.1_nodeMixUp0.6_edgeDrop0.1_lr0.005_epochs200_pcgrad'

result_names = ['Photo SAGE', 'Photo GCN', 'Photo SGC', 
                'CS SAGE', 'CS GCN', 'CS SGC', 
                'PubMed SAGE','PubMed GCN','PubMed SGC']

results_folders = [photo_sage_random_backdoor,photo_gcn_random_backdoor, photo_sgc_random_backdoor,
                   CS_sage_random_backdoor,CS_gcn_random_backdoor, CS_sgc_random_backdoor,
                   pubmed_sage_random_backdoor,pubmed_gcn_random_backdoor,pubmed_sgc_random_backdoor]

print("Watermark random backdoor")
for results_folder,results_name in zip(results_folders,result_names):
    if 'SAGE' in results_name:
        train_accs = []
        val_accs = []
        test_accs = []
        trigger_accs = []
        seeds = [f for f in os.listdir(results_folder) if 'seed' in f and f!='seed0']
        for s in seeds:
            file = results_folder + '/' + s + '/results_watermark_random_backdoor.txt'
            with open(file,'r') as f:
                lines = f.readlines()
            f.close()
            train_acc = float(lines[1].split("(trn/val/test)= ")[1].split('/')[0])
            val_acc = float(lines[1].split("(trn/val/test)= ")[1].split('/')[1])
            test_acc = float(lines[1].split("(trn/val/test)= ")[1].split('/')[2].split(', ')[0])
            trigger_acc = float(lines[1].split("trigger acc = ")[1].split('/')[0].split('\n')[0])
            train_accs.append(train_acc)
            val_accs.append(val_acc)
            test_accs.append(test_acc)
            trigger_accs.append(trigger_acc)
        print(f'{f"{results_name}:".ljust(11)} average train/val/test acc = {np.mean(train_accs):.3f}/{np.mean(val_accs):.3f}/{np.mean(test_accs):.3f}, average trigger acc = {np.mean(trigger_accs):.3f}')


### Graphlime Backdoor Watermark (Not our method)

In [ ]:

# WATERMARK GRAPHLIME BACKDOOR

# Examples of results you may want to display:
target_label=0
photo_sage_graphlime_backdoor  = training_folder + f'/photo/watermark_graphlime_backdoor_archSAGE_elu_nLayers3_hDim256_drop0.1_skipTruetargetLabel{target_label}_poisonRate0.05_watermarkSize0.2/watermark_graphlime_backdoor_nodeDropP0.3_nodeMixUp1_nodeFeatMask0.3_edgeDrop0.3_lr0.0002_epochs300_pcgrad'
photo_gcn_graphlime_backdoor   = training_folder + f'/photo/watermark_graphlime_backdoor_archGCN_elu_nLayers3_hDim256_drop0.1_skipTruetargetLabel{target_label}_poisonRate0.05_watermarkSize0.2/watermark_graphlime_backdoor_nodeDropP0.3_nodeMixUp1_nodeFeatMask0.3_edgeDrop0.3_lr0.0002_epochs300_pcgrad'
photo_sgc_graphlime_backdoor   = training_folder + f'/photo/watermark_graphlime_backdoor_archSGC_elu_nLayers3_hDim256_drop0.1_skipTruetargetLabel{target_label}_poisonRate0.05_watermarkSize0.2/watermark_graphlime_backdoor_nodeDropP0.3_nodeMixUp1_nodeFeatMask0.3_edgeDrop0.3_lr0.0002_epochs300_pcgrad'
CS_sage_graphlime_backdoor     = training_folder + f'/CS/watermark_graphlime_backdoor_archSAGE_elu_nLayers3_hDim256_drop0.1_skipTruetargetLabel{target_label}_poisonRate0.05_watermarkSize0.2/watermark_graphlime_backdoor_nodeDropP0.1_nodeMixUp5_edgeDrop0.1_lr0.001_epochs90_pcgrad'
CS_gcn_graphlime_backdoor      = training_folder + f'/CS/watermark_graphlime_backdoor_archGCN_elu_nLayers3_hDim256_drop0.1_skipTruetargetLabel{target_label}_poisonRate0.05_watermarkSize0.2/watermark_graphlime_backdoor_nodeDropP0.1_nodeMixUp5_edgeDrop0.1_lr0.001_epochs90_pcgrad'
CS_sgc_graphlime_backdoor      = training_folder + f'/CS/watermark_graphlime_backdoor_archSGC_elu_nLayers3_hDim256_drop0.1_skipTruetargetLabel{target_label}_poisonRate0.05_watermarkSize0.2/watermark_graphlime_backdoor_nodeDropP0.1_nodeMixUp5_edgeDrop0.1_lr0.001_epochs90_pcgrad'
pubmed_sage_graphlime_backdoor = training_folder + f'/PubMed/watermark_graphlime_backdoor_archSAGE_elu_nLayers3_hDim512_drop0.95_skipTruetargetLabel{target_label}_poisonRate0.05_watermarkSize0.2/watermark_graphlime_backdoor_nodeDropP0.1_nodeMixUp0.6_edgeDrop0.1_lr0.005_epochs200_pcgrad'
pubmed_gcn_graphlime_backdoor  = training_folder + f'/PubMed/watermark_graphlime_backdoor_archGCN_elu_nLayers3_hDim512_drop0.8_skipTruetargetLabel{target_label}_poisonRate0.05_watermarkSize0.2/watermark_graphlime_backdoor_nodeDropP0.1_nodeMixUp0.6_edgeDrop0.1_lr0.005_epochs200_pcgrad'
pubmed_sgc_graphlime_backdoor  = training_folder + f'/PubMed/watermark_graphlime_backdoor_archSGC_elu_nLayers3_hDim512_drop0.8_skipTruetargetLabel{target_label}_poisonRate0.05_watermarkSize0.2/watermark_graphlime_backdoor_nodeDropP0.1_nodeMixUp0.6_edgeDrop0.1_lr0.005_epochs200_pcgrad'
result_names = ['Photo SAGE', 'Photo GCN', 'Photo SGC', 
                'CS SAGE', 'CS GCN', 'CS SGC', 
                'PubMed SAGE','PubMed GCN','PubMed SGC']
results_folders = [photo_sage_graphlime_backdoor,photo_gcn_graphlime_backdoor, photo_sgc_graphlime_backdoor,
                   CS_sage_graphlime_backdoor,CS_gcn_graphlime_backdoor, CS_sgc_graphlime_backdoor,
                   pubmed_sage_graphlime_backdoor,pubmed_gcn_graphlime_backdoor,pubmed_sgc_graphlime_backdoor]
print("Watermark graphlime backdoor")
for results_folder,results_name in zip(results_folders,result_names):
    if 'SAGE' in results_name:
        train_accs = []
        val_accs = []
        test_accs = []
        backdoor_node_accs = []
        seeds = [f for f in os.listdir(results_folder) if 'seed' in f and f!='seed0']
        for s in seeds:
            file = results_folder + '/' + s + '/results_watermark_graphlime_backdoor.txt'
            with open(file,'r') as f:
                lines = f.readlines()
            f.close()
            train_acc = float(lines[1].split("(trn/val/test)= ")[1].split('/')[0])
            val_acc = float(lines[1].split("(trn/val/test)= ")[1].split('/')[1])
            test_acc = float(lines[1].split("(trn/val/test)= ")[1].split('/')[2].split(', ')[0])
            backdoor_node_acc = float(lines[1].split("backdoor nodes acc = ")[1].split('/')[0].split('\n')[0])
            train_accs.append(train_acc)
            val_accs.append(val_acc)
            test_accs.append(test_acc)
            backdoor_node_accs.append(backdoor_node_acc)
        print(f'{f"{results_name}:".ljust(11)} average train/val/test acc = {np.mean(train_accs):.3f}/{np.mean(val_accs):.3f}/{np.mean(test_accs):.3f}, backdoor_node_acc = {np.mean(backdoor_node_accs):.3f}')


In [ ]:
def get_performance_printout_for_subgraph_size(num_subgraph, subgraph_size, train_acc, val_acc, match_rate, p_val):
    avg_p_val_str = '<0.001' if p_val<0.001 else f'{p_val:.3f}'
    ret_str = f'{num_subgraph} SUBGRAPHS, SIZE {subgraph_size}:'
    ret_str += '\n   ACC_TRN/ACC_VAL'.ljust(25) + 'MATCH_RATE'.ljust(14) + 'P-VAL'
    ret_str += f'\n   {100*train_acc:.1f}/{100*val_acc:.1f}'.ljust(25) + f'{match_rate:.1f}'.ljust(14) + avg_p_val_str
    print(ret_str)

def get_latex_str_for_subgraph_size(train_acc, val_acc, match_rate, p_val):
    avg_p_val_str = '<0.001' if p_val<0.001 else f'{p_val:.3f}'
    return f'{100*train_acc:.1f}/{100*val_acc:.1f} & {match_rate:.1f} & {avg_p_val_str}'



# results_paths = [
# f'{training_folder}/photo/clf_only_archSAGE_elu_nLayers3_hDim256_drop0.1_skipTrue/clf_only_nodeDropP0.3_nodeMixUp1_nodeFeatMask0.3_edgeDrop0.3_lr0.0002_epochs300/seed0/results_clf_only.txt'
# ,
# f'{training_folder}/computers/clf_only_archGCN_elu_nLayers3_hDim256_drop0.1_skipTrue/seed0'
# ,
# f'{training_folder}/PubMed/clf_only_archSAGE_elu_nLayers3_hDim256_drop0.1_skipTrue/seed0'\
# ,
# f'{training_folder}/CS/clf_only_archSAGE_elu_nLayers3_hDim256_drop0.1_skipTrue/seed0'\
# ]

# # plot=True
# # for results_path in results_paths:
# #     dataset_name = (results_path.split('training_results')[1]).split('/')[0]
# #     lines = open(results_path,'r').readlines()
# #     print('dataset_name:',dataset_name)
# #     print(lines)
# #     size_lines = lines[1::2]
# #     value_lines = [v[:-1] if v[-1:]=='\n' else v for v in lines[2::2]]
# #     sizes = [int((line.split('(')[1]).split(' ')[0]) for line in size_lines]
# #     ps = [[float(entry.split(':')[1]) for entry in value.split(',')] for value in value_lines]
# #     sizes_sorted, ps_sorted = zip(*sorted(list(zip(sizes,ps))))
# #     size_p_dict = dict(zip(sizes_sorted,ps_sorted))



# #     if plot==True:

# #         p0s_sorted = [ps_[0] for ps_ in ps_sorted]
# #         p1s_sorted = [ps_[1] for ps_ in ps_sorted]
# #         pn1s_sorted = [ps_[2] for ps_ in ps_sorted]

# #         plt.plot(sizes_sorted, p0s_sorted,label='p(entry==0)')
# #         plt.plot(sizes_sorted, p1s_sorted,label='p(entry==1)')
# #         plt.plot(sizes_sorted, pn1s_sorted,label='p(entry==-1)')
# #         plt.xlabel('Subgraph Size (Number of Nodes)')
# #         plt.ylabel('Probability')
# #         plt.title(dataset_name)
# #         plt.legend()
# #         plt.show()
    

# with open(os.path.join(training_folder,'subgraph_size_comparison.txt'),'r') as f:
#     subgraph_sizes = ast.literal_eval(f.readline().split('subgraph_sizes: ')[1].split('\n')[0])
#     train_accs_ = ast.literal_eval(f.readline().split('train_accs: ')[1].split('\n')[0])
#     val_accs_ = ast.literal_eval(f.readline().split('val_accs: ')[1].split('\n')[0])
#     match_rates_ = ast.literal_eval(f.readline().split('match_rates: ')[1].split('\n')[0])
#     p_vals_ = ast.literal_eval(f.readline().split('p_values: ')[1])
# f.close()

In [ ]:
def extract_watermark_performance(dataset_name, seed, architecture_folder, num_subgraphs, subgraph_size, verification_confidence, continuation=False, starting_epoch=0, project_dir = ''):
    """
    Extracts and analyzes watermarking performance from trained model results.
    
    Args:
        dataset_name (str): Name of dataset (photo, CS, PubMed)
        seed (int): Random seed identifier for specific training run
        architecture_folder (str): Path to architecture-specific results
        num_subgraphs (int): Number of watermarked subgraphs used in training
        subgraph_size (float): Fraction of nodes per subgraph
        verification_confidence (float): Statistical confidence level (e.g., 0.99)
        continuation (bool): Whether this was continued training from checkpoint
        starting_epoch (int): Starting epoch if continuation training
        project_dir (str): Root project directory path
    
    Returns:
        tuple: (avg_trn_acc, avg_val_acc, avg_match_rate, avg_p_val)
            - avg_trn_acc: Average training accuracy across seeds
            - avg_val_acc: Average validation accuracy across seeds
            - avg_match_rate: Average watermark feature alignment rate
            - avg_p_val: Average p-value for watermark significance
        """
    data_training_path = f'{project_dir}/training_results/{dataset_name}/'
    if len(architecture_folder.split('/'))<3:
        architecture_folder = os.path.join(data_training_path, architecture_folder)

    train_val_test_split = [0.6,0.2,0.2]
    dataset = prep_data(dataset_name=dataset_name, location='default', batch_size='default', transform_list='default',  train_val_test_split=train_val_test_split, verbose=False)
    data = dataset[0]
    get_presets(dataset,dataset_name)
    config.subgraph_kwargs['numSubgraphs']=num_subgraphs
    
    def obtain_final_significance_against_empirical(Trainer_object, data, subgraph_dict_not_watermarked, mu_natural_empirical, sigma_natural_empirical):
        watermark_loss_kwargs = config_dict['watermark_loss_kwargs']
        optimization_kwargs = config_dict['optimization_kwargs']
        regression_kwargs = config_dict['regression_kwargs']
        node_classifier = Trainer_object.node_classifier
        watermark_match_rates, acc_trn, _, acc_val, target_matches, [_, match_count_wmk_without_zeros, _, confidence_wmk_without_zeros], \
            [_, _, _, _] = easy_run_node_classifier(Trainer_object, node_classifier, data, mu_natural_empirical, sigma_natural_empirical, subgraph_dict, subgraph_dict_not_watermarked, watermark_loss_kwargs, optimization_kwargs, regression_kwargs, target_confidence=verification_confidence,also_show_un_watermarked_counts=False)
        return acc_trn, acc_val, target_matches, match_count_wmk_without_zeros, confidence_wmk_without_zeros, watermark_match_rates
    
    model_paths = [os.path.join(architecture_folder, folder) for folder in os.listdir(architecture_folder) if folder[0]!='.' and 'ignore' not in folder]
    model_path = [p for p in model_paths if int(p.split('numSubgraphs')[1].split('_')[0])==num_subgraphs and float(p.split('fraction')[1].split('_')[0])==subgraph_size][0]
    assert os.path.exists(model_path)
    seeds = [seed_name for seed_name in os.listdir(model_path) if seed_name[0]!='.' and '.png' not in seed_name and 'ignore' not in seed_name]
    avg_acc_trn, avg_acc_val, avg_match_count_wmk_without_zeros, avg_confidence_wmk_without_zeros, avg_wmk_match_rates = [],[],[],[],[]
    avg_p_vals = []
    for seed in seeds:
        full_model_path = os.path.join(model_path,seed)
        assert os.path.exists(os.path.join(full_model_path,'config_dict'))
        config_dict = pickle.load(open(os.path.join(full_model_path,'config_dict'),'rb'))
        subgraph_dict_filename = 'subgraph_dict' if continuation==False else f'subgraph_dict_continuation_from_{starting_epoch}'
        subgraph_dict = pickle.load(open(os.path.join(full_model_path,subgraph_dict_filename),'rb'))
        Trainer_object_filename = 'Trainer' if continuation==False else f'Trainer_continuation_from_{starting_epoch}'
        Trainer_object = pickle.load(open(os.path.join(full_model_path,Trainer_object_filename),'rb'))
        distribution_records_filename = 'distribution.txt' if continuation==False else f'distribution_continuation_from_{starting_epoch}.txt'
        distribution_records_file = os.path.join(full_model_path,distribution_records_filename)
        with open(distribution_records_file,'r') as f:
            lines = f.readlines()
        f.close()
        mu_natural_empirical = float(lines[1].split('mu_natural=')[1].split(',')[0])
        sigma_natural_empirical = float(lines[1].split('sigma_natural=')[1])
        acc_trn, acc_val, _, match_count_wmk_without_zeros, confidence_wmk_without_zeros, watermark_match_rates = obtain_final_significance_against_empirical(Trainer_object, data, None, mu_natural_empirical, sigma_natural_empirical)
        avg_acc_trn.append(acc_trn.item())
        avg_acc_val.append(acc_val.item())
        avg_match_count_wmk_without_zeros.append(match_count_wmk_without_zeros)
        avg_confidence_wmk_without_zeros.append(confidence_wmk_without_zeros)
        avg_wmk_match_rates.append(watermark_match_rates)
        conf = np.mean(avg_confidence_wmk_without_zeros)
        avg_p_vals.append(1-conf)
    
    avg_trn_acc = np.mean(avg_acc_trn)
    avg_val_acc = np.mean(avg_acc_val)
    avg_match_rate = np.mean(watermark_match_rates)
    avg_p_val = np.mean(avg_p_vals)
    return avg_trn_acc, avg_val_acc,avg_match_rate, avg_p_val


 


def ablation_plot(dataset_hyperparameter_dict, datasets=['photo','PubMed','CS'], recollect=False, arch='SAGE', num_subgraph=4,exclude_title=False, variable='subgraph_size',
                                  xticksize=7,yticksize=7,xlabelsize=9,ylabelsize=9,smalltitlesize=11,bigtitlesize=13,legendtitlesize=7,annotatesize=7,xrotation=45, project_dir = ''):
    """
    Generates ablation study plots showing performance vs hyperparameter variations.
    
    Args:
        dataset_hyperparameter_dict (dict): Nested dict with dataset->architecture->hyperparameters
        datasets (list): List of dataset names to include in plot
        recollect (bool): Whether to recompute results or load from cache
        arch (str): GNN architecture to analyze (SAGE, GCN, SGC)
        num_subgraph (int): Number of subgraphs (used when variable='subgraph_size')
        exclude_title (bool): Whether to suppress plot title
        variable (str): Which hyperparameter to vary ('subgraph_size' or 'num_subgraph')
        **plot_kwargs: Additional plotting parameters (font sizes, etc.)
    
    Output: PNG figure showing performance trends across hyperparameter space
    """
    # project_dir: the folder that src, training_results, etc. are contained in
    # dropout assumed to be 0.1 unless new / different models are added, or if the dataset is PubMed
    # naming conventions may have shifted: nLayers vs numLayers

    starting_epoch=0
    subgraph_sizes = [0.000,0.002,0.004,0.006,0.008,0.010]
    num_subgraphs  = [2,3,4,5]
    num_subgraph=4
    subgraph_size=0.005
    fig = plt.figure(figsize=(8,1.5))
    gs = gridspec.GridSpec(1, 5, figure=fig,width_ratios=[1,0.1,1,0.1,1])
    for i, dataset_name in enumerate(datasets):
        train_results_dir = os.path.join(project_dir,'training_results',dataset_name)
        ax = fig.add_subplot(gs[0,i*2])

        dropout_str = dataset_hyperparameter_dict[dataset_name][arch]['dropout_str']
        numLayers = dataset_hyperparameter_dict[dataset_name][arch]['numLayers']
        hDim = dataset_hyperparameter_dict[dataset_name][arch]['hDim']
        preserve_edges_str = '_preserve_edges' if dataset_hyperparameter_dict[dataset_name][arch]['preserve_edges'] else ''
        
        architecture_folder = f'{train_results_dir}/arch{arch}_elu_nLayers{numLayers}_hDim{hDim}_drop{dropout_str}_skipTrue{preserve_edges_str}'

        verification_confidence=0.99

        if recollect==True:
            train_accs = []
            val_accs = []
            match_rates = []
            p_vals = []
            if variable=='subgraph_size':
                for subgraph_size in subgraph_sizes:
                    avg_trn_acc, avg_val_acc,avg_match_rate, avg_p_val = extract_watermark_performance(dataset_name, seed, architecture_folder,num_subgraphs=num_subgraph,subgraph_size=subgraph_size,verification_confidence=verification_confidence, continuation=False,starting_epoch=starting_epoch)
                    train_accs.append(avg_trn_acc)
                    val_accs.append(avg_val_acc)
                    match_rates.append(avg_match_rate)
                    p_vals.append(avg_p_val)  
                    get_performance_printout_for_subgraph_size(num_subgraph, subgraph_size, avg_trn_acc, avg_val_acc, avg_match_rate, avg_p_val)         
                    latex_str = get_latex_str_for_subgraph_size(avg_trn_acc, avg_val_acc, avg_match_rate, avg_p_val)
                    print('\nfor latex table:')
                    print(latex_str)
                with open(os.path.join(train_results_dir,'subgraph_size_comparison.txt'),'w') as f:
                    f.write(f'subgraph_sizes: {subgraph_sizes}')
                    f.write(f'\ntrain_accs: {train_accs}')
                    f.write(f'\nval_accs: {val_accs}')
                    f.write(f'\nmatch_rates: {match_rates}')
                    f.write(f'\np_values: {p_vals}')
                f.close()
            elif variable=='num_subgraph':
                for num_subgraph in num_subgraphs:
                    avg_trn_acc, avg_val_acc,avg_match_rate, avg_p_val = extract_watermark_performance(dataset_name, seed, architecture_folder,num_subgraphs=num_subgraph,subgraph_size=subgraph_size,verification_confidence=verification_confidence, continuation=False,starting_epoch=starting_epoch)
                    train_accs.append(avg_trn_acc)
                    val_accs.append(avg_val_acc)
                    match_rates.append(avg_match_rate)
                    p_vals.append(avg_p_val)  
                    get_performance_printout_for_subgraph_size(num_subgraph, subgraph_size, avg_trn_acc, avg_val_acc, avg_match_rate, avg_p_val)         
                    latex_str = get_latex_str_for_subgraph_size(avg_trn_acc, avg_val_acc, avg_match_rate, avg_p_val)
                    print('\nfor latex table:')
                    print(latex_str)
                with open(os.path.join(train_results_dir,'num_subgraph_comparison.txt'),'w') as f:
                    f.write(f'num_subgraphs: {num_subgraphs}')
                    f.write(f'\ntrain_accs: {train_accs}')
                    f.write(f'\nval_accs: {val_accs}')
                    f.write(f'\nmatch_rates: {match_rates}')
                    f.write(f'\np_values: {p_vals}')
                f.close()
        else:
            if variable=='subgraph_size':
                with open(os.path.join(train_results_dir,'subgraph_size_comparison.txt'),'r') as f:
                    subgraph_sizes = ast.literal_eval(f.readline().split('subgraph_sizes: ')[1].split('\n')[0])
                    train_accs = ast.literal_eval(f.readline().split('train_accs: ')[1].split('\n')[0])
                    val_accs = ast.literal_eval(f.readline().split('val_accs: ')[1].split('\n')[0])
                    match_rates = ast.literal_eval(f.readline().split('match_rates: ')[1].split('\n')[0])
                    p_vals = ast.literal_eval(f.readline().split('p_values: ')[1])
                f.close()
                for s,t,v,m,p in zip(subgraph_sizes,train_accs,val_accs,match_rates, p_vals):
                    get_performance_printout_for_subgraph_size(num_subgraph, s, t, v, m, p)         
                    latex_str = get_latex_str_for_subgraph_size(t, v, m, p)
                    print('\nfor latex table:')
                    print(latex_str)
            elif variable=='num_subgraph':
                with open(os.path.join(train_results_dir,'num_subgraph_comparison.txt'),'r') as f:
                    num_subgraphs = ast.literal_eval(f.readline().split('num_subgraphs: ')[1].split('\n')[0])
                    train_accs = ast.literal_eval(f.readline().split('train_accs: ')[1].split('\n')[0])
                    val_accs = ast.literal_eval(f.readline().split('val_accs: ')[1].split('\n')[0])
                    match_rates = ast.literal_eval(f.readline().split('match_rates: ')[1].split('\n')[0])
                    p_vals = ast.literal_eval(f.readline().split('p_values: ')[1])
                f.close()
                for n,t,v,m,p in zip(num_subgraphs,train_accs,val_accs,match_rates, p_vals):
                    get_performance_printout_for_subgraph_size(n, subgraph_size, t, v, m, p)         
                    latex_str = get_latex_str_for_subgraph_size(t, v, m, p)
                    print('\nfor latex table:')
                    print(latex_str)

        if variable=='num_subgraph':
            xvals=num_subgraphs
            default_str='Default:\n4 Subgraphs'
            xlabel='# Watermarked Subgraphs ($T$)'
            default_value=4
        elif variable=='subgraph_size':
            xvals=subgraph_sizes
            default_str='Default\nsize:\n0.005'
            xlabel=f'Watermarked Subgraph Size ($s$)'
            default_value=0.005

        ax2 = ax.twinx()
        ax.axvline(default_value,color='black',lw=0.5)
        ax.plot(xvals,[100*a for a in val_accs],color='blue',label='Test Acc',zorder=1) 
        ax.plot(xvals,match_rates,color='blue',linestyle='dashdot',label='Watermark Alignment',zorder=2)
        ax2.plot(xvals,p_vals,color='red',label='$p$-value',zorder=3,alpha=0.8)
        text_x = 0.0051 if variable=='subgraph_size' else 2.75
        ax.text(text_x,40,default_str,color='black',fontsize=annotatesize,zorder=10,clip_on=False)
        if variable=='subgraph_size':
            # Define the custom tick positions and labels
            custom_ticks = [0, 0.002,0.004,0.006,0.008,0.01]  # Fewer ticks than xvals
            custom_labels = ["0.000", "0.002", "0.004", "0.006", "0.008", "0.010"]  # Custom labels for these ticks
            # Apply the ticks and labels
            ax.set_xticks(custom_ticks)  # Set the tick positions
            ax.set_xticklabels(custom_labels, fontsize=xticksize, rotation=xrotation)  # Set the corresponding labels
        else:
            ax.set_xticks(xvals)
            ax.set_xticklabels(xvals,fontsize=xticksize,rotation=xrotation)

        acc_ticks = [0,20,40,60,80,100]
        ax.set_yticks(acc_ticks)
        ax.set_yticklabels(acc_ticks,fontsize=yticksize)
        ax2_yticks = [0, 1e-3, 2e-3, 3e-3]
        ax2_ytick_labels = ['0', '1e-3', '2e-3', '3e-3']
        ax2.set_yticks(ax2_yticks)
        ax2.set_yticklabels(ax2_ytick_labels,fontsize=yticksize)
        ax.set_ylim(0,102)
        ax2.set_ylim(-0.0001,0.003)
        ax.set_ylabel('Percent',fontsize=ylabelsize,labelpad=-38)
        ax.set_xlabel(xlabel,fontsize=xlabelsize)
        ax2.set_ylabel('p-value',fontsize=ylabelsize,labelpad=-33,rotation=270)
        ax.set_title(dataset_name.capitalize() if dataset_name=='photo' else dataset_name,fontsize=smalltitlesize)



    ax_handles, ax_labels = ax.get_legend_handles_labels()
    ax2_handles, ax2_labels = ax2.get_legend_handles_labels()
    handles = ax_handles + ax2_handles
    labels = ax_labels + ax2_labels
    legend_y=-0.45 if variable=='subgraph_size' else -0.3
    fig.legend(handles=handles, labels=labels, loc='center', bbox_to_anchor=(0.5, legend_y), ncol=3,fontsize=legendtitlesize)
    if exclude_title==False:
        fig.suptitle('Performance Metrics vs. Target Subgraph Size',y=1.1,fontsize=bigtitlesize)
    fig.subplots_adjust(wspace=0.35)
    plt.savefig(f'performance_vs_{variable}.png',bbox_inches='tight',dpi=300)
    plt.show()


# Example:

seed=0
num_subgraph=4
arch='SAGE'
xticksize=10#6.5
yticksize=10#6.5
xlabelsize=10#8
ylabelsize=10#8
smalltitlesize=14#11
bigtitlesize=13
legendtitlesize=10#7
annotatesize=10#7
exclude_title=True


try:
    current_dir = os.path.dirname(os.path.abspath(__file__))
except:
    current_dir = os.getcwd()
project_dir = os.path.dirname(current_dir)    


dataset_hyperparameter_dict = { 'photo':   {
                                            'SAGE':             {'dropout_str': '0.1',  'numLayers': 3,     'hDim': 256,    'preserve_edges': True},
                                            'GCN':              {'dropout_str': '0.1',  'numLayers': 3,     'hDim': 256,    'preserve_edges': False},
                                            'SGC':              {'dropout_str': '0.1',  'numLayers': 3,     'hDim': 256,    'preserve_edges': False},
                                            'Transformers':     {'dropout_str': '0.1',  'numLayers': 3,     'hDim': 256,    'preserve_edges': True}
                                            },
                                'CS':      {
                                            'SAGE':             {'dropout_str': '0.1',  'numLayers': 3,     'hDim': 256,    'preserve_edges': True},
                                            'GCN':              {'dropout_str': '0.1',  'numLayers': 3,     'hDim': 256,    'preserve_edges': False},
                                            'SGC':              {'dropout_str': '0.1',  'numLayers': 3,     'hDim': 256,    'preserve_edges': False},
                                            'Transformers':     {'dropout_str': '0.1',  'numLayers': 3,     'hDim': 256,    'preserve_edges': True}
                                            },
                                'PubMed':  {
                                            'SAGE':             {'dropout_str': '0.8',  'numLayers': 3,     'hDim': 512,    'preserve_edges': False},
                                            'GCN':              {'dropout_str': '0.95', 'numLayers': 3,     'hDim': 512,    'preserve_edges': False},
                                            'SGC':              {'dropout_str': '0.95', 'numLayers': 3,     'hDim': 512,    'preserve_edges': False},
                                            'Transformers':     {'dropout_str': '0.1',  'numLayers': 3,     'hDim': 256,    'preserve_edges': True}
                                            }
                                }


variable='num_subgraph'
xrotation=45 if variable=='subgraph_size' else 0
recollect=False
ablation_plot(dataset_hyperparameter_dict,  datasets=['photo','CS','PubMed'], recollect=recollect, arch=arch, num_subgraph=num_subgraph,exclude_title=exclude_title, variable=variable,
                            xticksize=xticksize,yticksize=yticksize,xlabelsize=xlabelsize,ylabelsize=ylabelsize,
                            smalltitlesize=smalltitlesize,bigtitlesize=bigtitlesize,legendtitlesize=legendtitlesize,annotatesize=annotatesize,xrotation=xrotation, project_dir=project_dir)
variable='subgraph_size'
xrotation=45 if variable=='subgraph_size' else 0
recollect=False
ablation_plot(dataset_hyperparameter_dict, datasets=['photo','CS','PubMed'], recollect=recollect, arch=arch, num_subgraph=num_subgraph,exclude_title=exclude_title, variable=variable,
                            xticksize=xticksize,yticksize=yticksize,xlabelsize=xlabelsize,ylabelsize=ylabelsize,
                            smalltitlesize=smalltitlesize,bigtitlesize=bigtitlesize,legendtitlesize=legendtitlesize,annotatesize=annotatesize,xrotation=xrotation, project_dir=project_dir)




In [ ]:

def binom(n, k):
    if k > n:
        return 0
    return math.comb(n, k)

def probability_at_least_j_same_group(N, k, c, n_sub, j):
    """
    Computes probability of problematic node overlap between watermarked subgraphs.
    
    Args:
        N (int): Total number of training nodes
        k (int): Total number of target feature positions (nodes × subgraphs)
        c (int): Number of watermarked subgraphs (collections)
        n_sub (int): Number of nodes per subgraph
        j (int): Minimum overlap size to consider problematic
    """
    # Each group size
    group_size = k // c
    total_ways = binom(N, n_sub)
    def prob_less_than_j_from_group():
        sum_prob = 0
        for m in range(j):
            sum_prob += binom(group_size, m) * binom(N - group_size, n_sub - m)
        return sum_prob / total_ways
    prob_no_group_j_or_more = prob_less_than_j_from_group() ** c
    return 1 - prob_no_group_j_or_more


def plot_overlap_probability(datasets = ['photo','PubMed','CS'], num_subgraph=4, xrotation=45,xticksize=7,yticksize=7,xlabelsize=8,ylabelsize=8,annotationsize=7,legendsize=8,legendtitlesize=8,bigtitlesize=13,smalltitlesize=11):

    """
    Analyzes and plots theoretical probability of subgraph node overlap.
    
    Args:
        datasets (list): Datasets to analyze
        num_subgraph (int): Number of watermarked subgraphs
        **style_kwargs: Plotting style parameters

    Purpose: Validates theoretical assumptions about subgraph independence and guides hyperparameter selection to avoid problematic overlap rates
    """
    subgraph_sizes = [0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.008,0.009,0.01]
    fig,axs=plt.subplots(1,3,figsize=(11,2.3))
    for i, dataset_name in enumerate(datasets):
        ax=axs[i]
        dataset = prep_data(dataset_name=dataset_name, location='default', batch_size='default', transform_list='default',  train_val_test_split=[0.6,0.2,0.2], verbose=False)
        data = dataset[0]
        len_train = len_train=sum(data.train_mask).item()

        subgraph_size_overlap_dicts={j:[] for j in [1,2,3,4,5,'n_sub']}
        for subgraph_size in subgraph_sizes:
            n_sub = int(subgraph_size*len_train)
            num_target_entries=  n_sub*4
            for j in [1,2,3,4,5,'n_sub']:
                j_ = n_sub if j=='n_sub' else j
                p_j_overlap = probability_at_least_j_same_group(len_train, num_target_entries, num_subgraph, n_sub, j_)
                subgraph_size_overlap_dicts[j].append(p_j_overlap)
        for j in [1,2,3,4,5,'n_sub']:
            j_key = j if j!='n_sub' else 'all'
            ax.plot(subgraph_sizes,subgraph_size_overlap_dicts[j],label=j_key)

        p_intervals=[0.0,0.2,0.4,0.6,0.8,1.0]
        ax.set_yticks(p_intervals)
        ax.set_yticklabels(p_intervals,fontsize=yticksize)
        ax.set_xticks(subgraph_sizes)
        ax.set_xticklabels(subgraph_sizes,fontsize=xticksize,rotation=xrotation)
        ax.text(0.0051,0.91,'Default Size\n0.005',color='black',fontsize=annotationsize)
        ax.set_ylabel(f'Probability',fontsize=ylabelsize,labelpad=1)
        ax.set_xlabel(f'Watermarked Subgraph Size ($s$)',fontsize=xlabelsize)
        ax.axvline(0.005,color='black',lw=0.5)
        ax.set_ylim(-0.05,1.05)
        ax.set_title('Photo' if dataset_name=='photo' else dataset_name,fontsize=smalltitlesize)
    handles, labels = ax.get_legend_handles_labels()
    legend = ax.legend(handles=handles, labels=labels, loc='center', bbox_to_anchor=(1.25, 0.5), ncol=1,fontsize=legendsize, title='# Nodes\nOverlapping',title_fontsize=legendtitlesize)
    legend.get_title().set_ha('center')
    fig.subplots_adjust(wspace=0.2)
    plt.savefig('probability_of_overlap.png',bbox_inches='tight',dpi=300)
    plt.show()


plot_overlap_probability(datasets=['photo','PubMed','CS'], num_subgraph=4, xrotation=45,xticksize=9,yticksize=9,xlabelsize=9,ylabelsize=9,annotationsize=8,legendsize=9,legendtitlesize=9,bigtitlesize=13,smalltitlesize=11)

In [ ]:
def gather_prune_stats(model_path, confidence, pruning_type='structured',ignore_zeros=True):
    """
    Extracts pruning attack results from saved experiment files.
    
    Args:
        model_path (str): Path to model results directory
        confidence (float): Statistical confidence level for significance testing
        pruning_type (str): Type of pruning applied ('structured' or 'unstructured')
        ignore_zeros (bool): Whether to exclude zero-valued features from match counting
    
    Returns:
        tuple: (prune_rates, train_accs, val_accs, wmk_matches, target_matches_empirical, 
                mu_natural_empirical, sigma_natural_empirical)
            - prune_rates: Array of pruning percentages tested
            - train_accs, val_accs: Accuracy curves during pruning
            - wmk_matches: Watermark match counts at each pruning level
            - target_matches_empirical: Significance threshold for detection
            - mu_natural_empirical, sigma_natural_empirical: Baseline distribution parameters
    """

    assert pruning_type in ['structured','unstructured']
    file = f'results_prune_{pruning_type}.txt'
    dataset_name = model_path.split('training_results')[1].split('/')[1].split('/')[0]
    all_train_accs = []
    all_val_accs = []
    all_wmk_matches = []
    all_target_matches_empirical = []
    all_mu_natural_empirical = []
    all_sigma_natural_empirical = []

    seeds = [path.split('seed')[1] for path in os.listdir(model_path) if path[0]!='.' and 'png' not in path]
    for seed in seeds:
        full_path = os.path.join(model_path,f'seed{seed}',file)
        with open(full_path,'r') as f:
            lines = f.readlines()
        f.close()
        prune_rates = [float(l_.split()[0]) for l_ in [l for l in ''.join(lines).split('Prune rate: ') if len(l)>0]]
        all_train_accs.append([float(l.split('Trn acc: ')[1].split()[0]) for l in lines[1:]])#[1::2]])
        all_val_accs.append([float(l.split('Val acc: ')[1].split()[0]) for l in lines[1:]])#[1::2]])
        if ignore_zeros==True:
            all_wmk_matches.append([int(l.split('# Matches Wmk w/wout 0s:')[1].split()[0][1:-1].split(',')[1]) for l in [line for line in lines if "# Matches" in line]])
        elif ignore_zeros==False:
            all_wmk_matches.append([int(l.split('# Matches Wmk w/wout 0s:')[1].split()[0][1:-1].split(',')[0]) for l in [line for line in lines if "# Matches" in line]])
        
        mu_natural_empirical = float(''.join(open(os.path.join(model_path,f'seed{seed}','distribution.txt')).readlines()).split('mu_natural=')[1].split(',')[0])
        sigma_natural_empirical = float(''.join(open(os.path.join(model_path,f'seed{seed}','distribution.txt')).readlines()).split('sigma_natural=')[1].split(',')[0])
        z_t = norm.ppf(confidence)
        all_mu_natural_empirical.append(mu_natural_empirical)
        all_sigma_natural_empirical.append(sigma_natural_empirical)       
        target_matches_empirical = np.ceil(min(mu_natural_empirical +z_t*sigma_natural_empirical,config.dataset_attributes[dataset_name]['num_features']))
        all_target_matches_empirical.append(target_matches_empirical) 

    train_accs = torch.mean(torch.vstack([torch.tensor(l) for l in all_train_accs]),dim=0)
    val_accs = torch.mean(torch.vstack([torch.tensor(l) for l in all_val_accs]),dim=0)
    wmk_matches = torch.mean(torch.vstack([torch.tensor(l) for l in all_wmk_matches]),dtype=float,dim=0)
    mu_natural_empirical = int(np.round(np.mean(all_mu_natural_empirical)))
    sigma_natural_empirical = int(np.round(np.mean(all_sigma_natural_empirical)))
    target_matches_empirical = int(np.round(np.mean(all_target_matches_empirical)))

    return prune_rates, train_accs, val_accs, wmk_matches, target_matches_empirical, mu_natural_empirical, sigma_natural_empirical

def gather_fine_tune_stats(model_path, confidence, ignore_zeros=True, lr=None, lr_scale=None, seeds=[1,2,3,4]):
    """
    Extracts fine-tuning attack results from saved experiment files. Analyzes how fine-tuning attacks degrade watermark detectability over time.
    
    Args:
        model_path (str): Path to model results directory
        confidence (float): Statistical confidence level
        ignore_zeros (bool): Whether to exclude zero features from analysis
        lr (float): Explicit learning rate used (mutually exclusive with lr_scale)
        lr_scale (float): Learning rate as fraction of original (mutually exclusive with lr)
        seeds (list): Which experimental seeds to include in analysis
    
    Returns:
        tuple: (epochs, wmk_matches, mu_natural_empirical, sigma_natural_empirical, target_matches_empirical, og_trn_accs, new_trn_accs, val_accs)
            - epochs: Array of fine-tuning epoch numbers
            - wmk_matches: Watermark detection strength over time
            - Natural distribution parameters for significance testing
            - og_trn_accs: Accuracy on original training data
            - new_trn_accs: Accuracy on fine-tuning data
            - val_accs: Validation accuracy during fine-tuning
    """
    if lr is not None:
        assert lr_scale is None
        file = f'results_fine_tune_lr={float(lr)}.txt' if lr is not None else 'results_fine_tune.txt' # if continuation==False else f'results_fine_tune_continuation_from_{starting_epoch}.txt'
    elif lr_scale is not None:
        assert lr is None
        file = f'results_fine_tune_lr_scale={float(lr_scale)}.txt' if lr_scale is not None else 'results_fine_tune.txt' # if continuation==False else f'results_fine_tune_continuation_from_{starting_epoch}.txt'
    else:
        file = 'results_fine_tune.txt'

    dataset_name = model_path.split('training_results')[1].split('/')[1].split('/')[0]
    all_wmk_matches = []
    seeds_ = [path.split('seed')[1] for path in os.listdir(model_path) if path[0]!='.' and 'png' not in path]# and int(path.split('seed')[1]) in seeds]
    if seeds is not None and not seeds_:
        seeds_ = [s for s in seeds_ if int(s) in seeds]

    all_target_matches_empirical = []
    all_mu_natural_empirical = []
    all_sigma_natural_empirical = []
    og_trn_accs = []
    new_trn_accs = []
    val_accs = []


    for seed in seeds_:
        full_path = os.path.join(model_path,f'seed{seed}',file)
        with open(full_path,'r') as f:
            lines = f.readlines()
        f.close()

        epochs = [int(l.split('Epoch:')[1].split(',')[0].split()[0]) for l in lines]
        if ignore_zeros==True:
            l1 = [int(l.split('#_match_wmk w/wout 0s =')[1].split(', confidence')[0].split()[0][1:-1].split(',')[1]) for l in lines]
            all_wmk_matches.append(l1)
        elif ignore_zeros==False:
            l1=[int(l.split('#_match_wmk w/wout 0s =')[1].split(', confidence')[0].split()[0][1:-1].split(',')[0]) for l in lines]
            all_wmk_matches.append(l1)

        og_trn_accs.append([float(l.split('train (og) acc = ')[1].split(',')[0]) for l in lines])
        new_trn_accs.append([float(l.split('train (fine-tune) acc = ')[1].split(',')[0]) for l in lines])
        val_accs.append([float(l.split('val acc = ')[1].split(',')[0]) for l in lines])


        mu_natural_empirical = float(''.join(open(os.path.join(model_path,f'seed{seed}','distribution.txt')).readlines()).split('mu_natural=')[1].split(',')[0])
        sigma_natural_empirical = float(''.join(open(os.path.join(model_path,f'seed{seed}','distribution.txt')).readlines()).split('sigma_natural=')[1].split(',')[0])
        z_t = norm.ppf(confidence)
        all_mu_natural_empirical.append(mu_natural_empirical)
        all_sigma_natural_empirical.append(sigma_natural_empirical)       
        target_matches_empirical = np.ceil(min(mu_natural_empirical +z_t*sigma_natural_empirical,config.dataset_attributes[dataset_name]['num_features']))
        all_target_matches_empirical.append(target_matches_empirical) 

    wmk_matches = torch.mean(torch.vstack([torch.tensor(l) for l in all_wmk_matches]),dtype=float,dim=0)
    mu_natural_empirical = int(np.round(np.mean(all_mu_natural_empirical)))
    sigma_natural_empirical = int(np.round(np.mean(all_sigma_natural_empirical)))
    target_matches_empirical = int(np.round(np.mean(all_target_matches_empirical)))
    og_trn_accs = torch.mean(torch.vstack([torch.tensor(l) for l in og_trn_accs]),dtype=float,dim=0)
    new_trn_accs = torch.mean(torch.vstack([torch.tensor(l) for l in new_trn_accs]),dtype=float,dim=0)
    val_accs = torch.mean(torch.vstack([torch.tensor(l) for l in val_accs]),dtype=float,dim=0)

    return epochs, wmk_matches, mu_natural_empirical, sigma_natural_empirical, target_matches_empirical, og_trn_accs, new_trn_accs, val_accs


def arrow(ax, target_matches):
    x_start = 0.05
    trans = transforms.blended_transform_factory(ax.transAxes,ax.transAxes)
    if target_matches > ax.get_ylim()[1]:
        y_start  = 0.85#y_ub-0.1
        dy = 0.05
    elif target_matches < ax.get_ylim()[0]: 
        y_start = 0.15#y_lb+0.1
        dy = -0.05
    arrow = patches.FancyArrow(x_start, y_start, 0, dy, transform=trans,  color='r', width=0.02,  head_width=0.05, head_length=0.06)
    ax.add_patch(arrow)



def arrow_off_plot(ax, curve):
    x_start = 0.05
    trans = transforms.blended_transform_factory(ax.transAxes,ax.transAxes)
    if sum([c > ax.get_ylim()[1] for c in curve])==len(curve):
        y_start  = 0.85
        dy = 0.05
        arrow = patches.FancyArrow(x_start, y_start, 0, dy, transform=trans,  color='r', width=0.02,  head_width=0.05, head_length=0.06)
        ax.add_patch(arrow)
        return True
    return False

def prune_plot(prune_rates, train_accs, val_accs, wmk_matches, c, mu_natural, sigma_natural, target_matches, axs=None,
               xlabelsize=8, xticksize=8,yticksize=8,ylabelsize=8,bigtitlesize=13,smalltitlesize=11, ablation=False, ablation_value=None, ablation_color='blue', dataset_name='<insert_dataset_name_here>'):
    """
    Creates publication-ready plots showing watermark persistence under pruning attacks.
    
    Args:
        prune_rates (array): X-axis values (pruning percentages)
        train_accs, val_accs (array): Training/validation accuracy curves
        wmk_matches (array): Watermark match counts at each pruning level
        c (int): Number of subgraphs (for title)
        mu_natural, sigma_natural (float): Natural distribution parameters
        target_matches (float): Significance threshold for detection
        axs (list): Optional pre-created subplot axes
        **style_kwargs: Font sizes and styling parameters
    
    Output: Subplot showing how pruning affects both accuracy and watermark detectability
    
    Note: Converts raw match counts to p-values for intuitive interpretation
    """
    exclude_accuracies = True if len(axs)==1 else False
    if ablation==True:
        assert ablation_value is not None

    def scale_y1_to_pvalue(y1_value, mu_natural, sigma_natural):
        z_score = (y1_value - mu_natural)/sigma_natural
        p_value = 1-stats.norm.cdf(z_score)
        return p_value


    if exclude_accuracies==True:
        if axs is None:
            fig, ax = plt.subplots(1, 1, figsize=(3, 4))
            pval_ax = ax
        else:
            fig = None
            pval_ax = axs[0]

    elif exclude_accuracies==False:
        if axs is None:
            fig, axs = plt.subplots(2, 1, figsize=(3, 4))
            acc_ax = axs[0]
            pval_ax = axs[1]
        else:
            fig = None
            acc_ax, pval_ax = axs[0], axs[1]
    target_matches=0.95
    wmk_matches = torch.tensor([scale_y1_to_pvalue(num_matches, mu_natural, sigma_natural) for num_matches in wmk_matches])

    if exclude_accuracies==False:
        if ablation==True:
            acc_ax.plot(prune_rates, val_accs, label='Test Acc',color=ablation_color,linestyle='solid') # switch names of "test" and "validation" for paper results
        else:
            acc_ax.plot(prune_rates, train_accs, label='Train Acc',color='blue',linestyle='solid')
            acc_ax.plot(prune_rates, val_accs, label='Test Acc',color='blue',linestyle='dotted') # switch names of "test" and "validation" for paper results

    label=ablation_value if ablation==True else 'MI $p$-value'
    pval_curve_color = 'green' if ablation==False else ablation_color
    pval_ax.plot(prune_rates, wmk_matches, label=label,color=pval_curve_color)
    x_lb = -0.05
    x_ub = max(prune_rates)#0.95


    y_min = int(np.floor(torch.min(wmk_matches).item()))
    y_max = int(np.ceil(torch.max(wmk_matches).item()))
    if target_matches >= y_min and target_matches <= y_max:
        pval_ax.axhline(target_matches, color='red',linestyle=':',linewidth=1)
        pval_ax.axhline(0,linewidth=0.3,color='gray')
        pval_ax.axhline(0.01,linewidth=0.3,color='gray')
        pval_ax.axhline(0.05,linewidth=0.3,color='gray')
        pval_ax.axhline(0.1,linewidth=0.3,color='gray')
        pval_ax.axhline(0.2,linewidth=0.3,color='gray')
    else:
        arrow(pval_ax, target_matches)
        y_text_pos = y_min+0.9*(y_max-y_min) if target_matches>y_min else y_min+0.1*(y_max-y_min)
        x_text_pos = x_lb+0.1*(x_ub-x_lb)
        pval_ax.text(x_text_pos,y_text_pos, f'Target # Matches = {int(target_matches)}')


    if fig:    
        fig.suptitle('Effect of Model Pruning on Performance',y=1.01, fontsize=bigtitlesize)
        fig.text(0.5, 0.9, f'("{dataset_name.capitalize()}", {c} Subgraphs)', ha='center', fontsize=smalltitlesize)
    if len(prune_rates)>20:
        prune_rate_xticks = prune_rates[:-1][::10]
    else:
        prune_rate_xticks = prune_rates[:-1]

    if exclude_accuracies==False:
        accuracy_yticks = [0,0.2,0.4,0.6,0.7,0.8,1]
        acc_ax.set_ylim(0,1.01)
        y_lb = acc_ax.get_ylim()[0]
        y_lb_index = accuracy_yticks.index(y_lb)
        accuracy_yticks = accuracy_yticks[y_lb_index+1:]
        acc_ax.set_yticks(accuracy_yticks)
        acc_ax.set_xlabel('Prune Rate',fontsize=xlabelsize)
        acc_y_label='Accuracy' if ablation==False else 'Test Acc' # switch names of "test" and "validation" for paper results
        acc_ax.set_ylabel(acc_y_label,fontsize=ylabelsize)
        acc_ax.set_xlim(-0.05,x_ub)
        acc_ax.set_xticks(prune_rate_xticks)
        acc_ax.set_xticklabels(prune_rate_xticks,fontsize=xticksize)
        acc_ax.set_yticks(accuracy_yticks)
        acc_ax.set_yticklabels(accuracy_yticks,fontsize=yticksize)

    p_val_yticks = [0.01,0.05,0.1,0.2]
    pval_ax.set_xlabel('Prune Rate',fontsize=xlabelsize)
    pval_ax.set_ylabel('$p$-value',fontsize=ylabelsize)
    pval_ax.set_xlim(x_lb, x_ub)
    pval_ax.set_ylim(-0.01,0.21)
    pval_ax.set_xticks(prune_rate_xticks)
    pval_ax.set_xticklabels(prune_rate_xticks,fontsize=xticksize)
    pval_ax.set_yticks(p_val_yticks)
    pval_ax.set_yticklabels(p_val_yticks,fontsize=yticksize)
    val_off_plot = arrow_off_plot(pval_ax, wmk_matches)
    return val_off_plot

def fine_tune_plot(dataset_name, epochs, wmk_matches, c, mu_natural, sigma_natural, target_matches, og_trn_accs, new_trn_accs, val_accs, model_path, axs=None,
                   xlabelsize=8, xticksize=8,yticksize=8,ylabelsize=8,bigtitlesize=13,smalltitlesize=11, ablation=False, ablation_value=None, ablation_color='blue'):
    """
    Creates plots showing watermark degradation during fine-tuning attacks.
    
    Args:
        dataset_name (str): Dataset name for plot title
        epochs (array): X-axis values (fine-tuning epoch numbers)
        wmk_matches (array): Watermark detection strength over time
        c (int): Number of subgraphs used
        mu_natural, sigma_natural (float): Baseline distribution parameters
        target_matches (float): Statistical significance threshold
        og_trn_accs (array): Accuracy on original training data over time
        new_trn_accs (array): Accuracy on new fine-tuning data
        val_accs (array): Validation accuracy during fine-tuning
        model_path (str): Path for saving plot
        axs (list): Optional pre-created axes
    """
    print('mu_natural, sig_natural:',mu_natural, sigma_natural)
    def scale_y1_to_pvalue(y1_value, mu_natural, sigma_natural):
        z_score = (y1_value - mu_natural)/sigma_natural
        p_value = 1-stats.norm.cdf(z_score)
        return p_value

    exclude_accuracies=True if len(axs)==1 else False
    target_matches=0.95
    print('num_matches:',wmk_matches)
    wmk_matches = torch.tensor([scale_y1_to_pvalue(num_matches, mu_natural, sigma_natural) for num_matches in wmk_matches])

    if ablation==True:
        assert ablation_value is not None
        label=ablation_value
    elif ablation==False:
        label='MI $p$-value'
    
    if exclude_accuracies==True:
        if axs is None:
            fig, ax = plt.subplots(2, 1, figsize=(3, 4))
            pval_ax = ax
        else:
            fig=None
            pval_ax = axs[0]
    elif exclude_accuracies==False:
        if axs is None:
            fig, axs = plt.subplots(2, 1, figsize=(3, 4))
            acc_ax = axs[0]
            pval_ax = axs[1]
        else:
            fig = None
            acc_ax, pval_ax = axs[0], axs[1]

    if exclude_accuracies==False:
        if ablation==False:
            acc_ax.plot(epochs, new_trn_accs, label='Train Acc (New Data)',color='blue',linestyle='solid') 
            acc_ax.plot(epochs, og_trn_accs, label='Train Acc (Original Data)',color='blue',linestyle='dashed') 
            acc_ax.plot(epochs, val_accs, label='Test Acc',color='blue',linestyle='dotted') # switch names of "test" and "validation" for paper results
        else:
            acc_ax.plot(epochs, val_accs, label='Test Acc',color=ablation_color,linestyle='solid')  # switch names of "test" and "validation" for paper results

    pval_curve_color = ablation_color if ablation==True else 'green'
    pval_ax.plot(epochs, wmk_matches, label=label,color=pval_curve_color)

    x_lb = 0
    x_ub = epochs[-1]+1
    y_min = int(np.floor(torch.min(wmk_matches).item()))
    y_max = int(np.ceil(torch.max(wmk_matches).item()))
    if target_matches >= y_min and target_matches <= y_max:
        pval_ax.axhline(0,linewidth=0.3,color='gray')
        pval_ax.axhline(0.01,linewidth=0.3,color='gray')
        pval_ax.axhline(0.05,linewidth=0.3,color='gray')
        pval_ax.axhline(0.1,linewidth=0.3,color='gray')
        pval_ax.axhline(0.2,linewidth=0.3,color='gray')
    else:
        arrow(pval_ax, target_matches)
        y_text_pos = y_min+0.9*(y_max-y_min) if target_matches>y_min else y_min+0.1*(y_max-y_min)
        x_text_pos = x_lb+0.1*(x_ub-x_lb)
        pval_ax.text(x_text_pos,y_text_pos, f'Target # Matches = {int(target_matches)}')
    if fig:
        fig.suptitle('Effect of Model Fine-Tuning on Performance',y=1.01, fontsize=bigtitlesize)
        fig.text(0.5, 0.9, f'("{dataset_name.capitalize()}", {c} Subgraphs)', ha='center', fontsize=smalltitlesize)
        image_path = os.path.join(model_path,'fine_tuning_plot.png')
        plt.savefig(image_path)
        plt.show()

    epoch_xticks = [0,10,20,30,40]

    if exclude_accuracies==False:
        acc_ax.set_xlabel('Fine-Tuning Epoch',fontsize=xlabelsize)
        acc_y_label='Accuracy' if ablation==False else 'Test Acc' # switch names of "test" and "validation" for paper results
        acc_ax.set_ylabel(acc_y_label,fontsize=ylabelsize)
        acc_ax.set_ylim(0.7,1.01)
        accuracy_yticks = [0.4,0.5,0.6,0.7,0.75,0.8,0.85,0.9,0.95,1.00]
        y_lb = acc_ax.get_ylim()[0]
        y_lb_index = accuracy_yticks.index(y_lb)
        accuracy_yticks = accuracy_yticks[y_lb_index+1:]
        acc_ax.set_yticks(accuracy_yticks)
        acc_ax.set_xlim(x_lb, x_ub)
        acc_ax.set_yticklabels(accuracy_yticks,fontsize=yticksize)
        acc_ax.set_xticks(epoch_xticks)

    p_val_yticks = [0.01,0.05,0.1,0.2]
    pval_ax.set_xlabel('Fine-Tuning Epoch',fontsize=xlabelsize)
    pval_ax.set_ylabel('$p$-value',fontsize=ylabelsize)
    pval_ax.set_xlim(x_lb, x_ub)
    pval_ax.set_ylim(-0.01,0.21)
    pval_ax.set_yticks(p_val_yticks)
    pval_ax.set_yticklabels(p_val_yticks,fontsize=yticksize)
    pval_ax.set_xticks(epoch_xticks)
    pval_ax.set_xticklabels(epoch_xticks,fontsize=xticksize)
    val_off_plot = arrow_off_plot(pval_ax, wmk_matches)
    return val_off_plot



In [ ]:
def parse_num_subgraphs_from_path(s):
    # For example, return the length of each string
    return int(s.split('numSubgraphs')[1].split('_')[0])

def parse_subgraph_size(f):
    return float(f.split('fraction')[1].split('_')[0])


def plot_all_datasets_ablation(dataset_architecture_dict, variable='subgraph_size',task='prune',pruning_type='unstructured', arch='SAGE',exclude_title=False, include_accs=False,show_subgraph_sizes='all',
                              xrotation=45,xticksize=7,yticksize=7,xlabelsize=9,ylabelsize=9,bigtitlesize=13,smalltitlesize=11,legendsize=8, lr_scale=0.1, seeds=[0,1,2,3,4], project_dir = ''):
    """
    Creates comprehensive ablation study plots across multiple datasets and conditions.
    
    Args:
        dataset_architecture_dict (dict): Dataset names mapped to architectures
        variable (str): Hyperparameter to vary ('subgraph_size' or 'num_subgraph')
        task (str): Type of analysis ('prune' or 'fine_tune')
        pruning_type (str): Pruning method if task='prune'
        arch (str): GNN architecture to analyze
        include_accs (bool): Whether to show accuracy plots alongside p-values
        **style_kwargs: Detailed styling parameters
    """

    num_cols = len(dataset_architecture_dict)
    fig = plt.figure(figsize=(6, 2.5))
    gs = gridspec.GridSpec(2, num_cols, figure=fig)
    if variable=='subgraph_size':
        if show_subgraph_sizes=='all':
            subgraph_sizes = [0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.008,0.009,0.01]
        elif show_subgraph_sizes=='evens':
            subgraph_sizes = [0.002,0.004,0.006,0.008,0.01]
        elif show_subgraph_sizes=='odds':
            subgraph_sizes = [0.001,0.003,0.005,0.007,0.009]
    num_subgraphs = [2,3,4,5]
    subgraph_size=0.005
    num_subgraph=4
    value_list = subgraph_sizes if variable=='subgraph_size' else num_subgraphs
    color_cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']

    for i, dataset_name in enumerate(dataset_architecture_dict.keys()):

        training_results_folder = f'{project_dir}/training_results/{dataset_name}'
        architecture_folder = [os.path.join(training_results_folder,f) for f in os.listdir(training_results_folder) if f[0]!='.' and 'clf' not in f and 'watermark' not in f and 'KD' not in f and arch in f][0]
        model_folders = [f for f in os.listdir(architecture_folder) if f[0]!='.']
        confidence = 0.95
        col = i
        if include_accs==False:
            pval_ax = fig.add_subplot(gs[0,col])
            axs = [pval_ax]
        if include_accs==True:
            acc_ax = fig.add_subplot(gs[0,col])
            pval_ax = fig.add_subplot(gs[1,col])
            axs = [acc_ax, pval_ax]

        
        off_plot_vals = []
        for j, value in enumerate(value_list):
            if variable=='subgraph_size':
                subgraph_size=value
            elif variable=='num_subgraph':
                num_subgraph=value

            ablation_color = color_cycle[j]
            print(f"looking for model folder with subgraph_size={subgraph_size} and num_subgraph={num_subgraph}")
            model_folder = [os.path.join(architecture_folder,f) for f in model_folders if parse_subgraph_size(f)==subgraph_size and parse_num_subgraphs_from_path(f)==num_subgraph][0]
            c = int(model_folder.split('numSubgraphs')[1].split('_')[0])


            if task=='prune':
                prune_rates, train_accs, val_accs, wmk_matches, target_matches_empirical, mu_natural_empirical, sigma_natural_empirical = gather_prune_stats(model_folder, confidence,pruning_type=pruning_type)
                val_off_plot = prune_plot(prune_rates, train_accs, val_accs, wmk_matches, c, mu_natural_empirical, sigma_natural_empirical, target_matches_empirical, axs=axs,
                        xlabelsize=xlabelsize, xticksize=xticksize,yticksize=yticksize,ylabelsize=ylabelsize,bigtitlesize=bigtitlesize,smalltitlesize=smalltitlesize, ablation=True, ablation_value=value, ablation_color=ablation_color)
                if val_off_plot==True:
                    off_plot_vals.append(value)

            elif task=='fine_tune':
                epochs, wmk_matches, mu_natural_empirical, sigma_natural_empirical, target_matches_empirical, og_trn_accs, new_trn_accs, val_accs = gather_fine_tune_stats(model_folder, confidence, ignore_zeros=True, lr_scale=lr_scale, seeds=seeds)#, continuation=continuation,starting_epoch=starting_epoch)
                val_off_plot= fine_tune_plot(dataset_name, epochs, wmk_matches, c, mu_natural_empirical, sigma_natural_empirical, target_matches_empirical, og_trn_accs, new_trn_accs, val_accs, model_folder, axs=axs,
                            xlabelsize=xlabelsize, xticksize=xticksize,yticksize=yticksize,ylabelsize=ylabelsize,bigtitlesize=bigtitlesize,smalltitlesize=smalltitlesize, ablation=True, ablation_value=value, ablation_color=ablation_color)
                if val_off_plot==True:
                    off_plot_vals.append(value)

            if task=='prune':
                if pruning_type=='unstructured':
                    xticks = np.linspace(0,1,6)[:-1]
                    xticklabels = [f'{v:.2f}' for v in xticks]
                    x_lb,x_ub = -0.01,1.01
                elif pruning_type=='structured':
                    xticks = np.linspace(0,0.5,6)[:-1]
                    xticklabels = [f'{v:.2f}' for v in xticks]
                    x_lb,x_ub = -0.01,0.501
            elif task=='fine_tune':
                xticks = xticklabels = [int(i) for i in np.linspace(0,50,6)[:-1]]
                x_lb,x_ub = -1,51

            pval_ax.set_label(value)
            if include_accs==True:
                acc_ax.set_title(f'Photo' if dataset_name=='photo' else f'{dataset_name}',fontsize=smalltitlesize)
            else:
                pval_ax.set_title(f'Photo' if dataset_name=='photo' else f'{dataset_name}',fontsize=smalltitlesize)
            pval_ax.set_xticks(xticks)
            pval_ax.set_xticklabels(xticklabels,rotation=xrotation,fontsize=xticksize)
            pval_ax.set_xlim(x_lb,x_ub)
        if i==0:
            handles_1_, labels_1_ = [],[]
            handles_1, labels_1 = pval_ax.get_legend_handles_labels()
            handles_1_.extend(handles_1)
            labels_1_.extend(labels_1)
            if include_accs==True:
                handles_0_, labels_0_ = [],[]
                handles_0, labels_0 = acc_ax.get_legend_handles_labels()
                handles_0_.extend(handles_0)
                labels_0_.extend(labels_0)

        pval_ax.label_outer()
        if include_accs==True:
            acc_ax.label_outer()
        if len(off_plot_vals)>0:
            x_text_loc = 0.05 if task=='prune' else 5
            var_letter = "$s$" if variable=='subgraph_size' else "$c$"
            pval_ax.text(x_text_loc,0.90*0.2,f"{var_letter} = {','.join([str(v) for v in off_plot_vals])}",bbox=dict(facecolor='lightgray', alpha=0.7,edgecolor='none',pad=0.1),color='black')

    if exclude_title==False:
        if task=='prune':
            if pruning_type=='unstructured':
                fig.suptitle(f'{pruning_type.capitalize()} Pruning',y=1.02,fontsize=bigtitlesize)
            else:
                fig.suptitle(f'Pruning',y=1.02,fontsize=bigtitlesize)
        elif task=='fine_tune':
            fig.suptitle('Fine Tuning',y=1.02,fontsize=bigtitlesize)
    task_descrip = f'{pruning_type}_pruning' if task=='prune' else 'fine_tuning'
    image_name = f'all_datasets_{arch}_{task_descrip}_ablation_{variable}'
    ncols_legend=4 if variable=='subraph_size' else 5
    ylegend=-0.15 if task=='prune' else -0.15
    legend_title = 'Number Watermarked Subgraphs ($T$)' if variable=='num_subgraph' else 'Watermarked Subgraph Size ($s$)'
    fig.legend(handles=handles_1_,labels=labels_1_,ncols=ncols_legend, loc='center', bbox_to_anchor=(0.5,ylegend),fontsize=legendsize,title=legend_title)
    fig.subplots_adjust(wspace=0.0,hspace=0.0)

    plt.savefig(f'{project_dir}/{image_name}.png',bbox_inches='tight',dpi=300)


def plot_all_datasets(dataset_architecture_dict, num_subgraphs, subgraph_size, task='prune',pruning_type='unstructured', arch='SAGE',exclude_title=False,
                              xrotation=45,xticksize=7,yticksize=7,xlabelsize=9,ylabelsize=9,bigtitlesize=13,smalltitlesize=11,legendsize=8, lr=None, lr_scale=None, seeds=[1,2,3,4], projet_dir=''):
    """
    Creates standardized comparison plots across datasets for fixed hyperparameters.
    
    Args:
        dataset_architecture_dict (dict): Dataset configurations
        num_subgraphs (int): Fixed number of watermarked subgraphs
        subgraph_size (float): Fixed subgraph size fraction
        task (str): Analysis type ('prune' or 'fine_tune')
        **kwargs: Additional parameters (learning rates, styling, etc.)
    """
    
    if lr is not None:
        assert lr_scale is None
    if lr_scale is not None:
        assert lr is None

    num_cols = len(dataset_architecture_dict)
    fig = plt.figure(figsize=(6, 2.5))
    gs = gridspec.GridSpec(2, num_cols, figure=fig)
    for i, dataset_name in enumerate(dataset_architecture_dict.keys()):
        print(dataset_name,arch, num_subgraphs, subgraph_size)
        training_results_folder = f'{project_dir}/training_results/{dataset_name}'
        architecture_folder = [os.path.join(training_results_folder,f) for f in os.listdir(training_results_folder) if f[0]!='.' and 'clf' not in f and 'watermark' not in f and 'KD' not in f and arch in f][0]
        model_folders = [f for f in os.listdir(architecture_folder) if f[0]!='.']
        print(f"looking for model with subgraph_size={subgraph_size} and num_subgraphs={num_subgraphs}")
        model_folder = [os.path.join(architecture_folder,f) for f in model_folders if parse_subgraph_size(f)==subgraph_size and parse_num_subgraphs_from_path(f)==num_subgraphs][0]
        col = i
        ax1 = fig.add_subplot(gs[0,col])
        ax2 = fig.add_subplot(gs[1,col])
        c = int(model_folder.split('numSubgraphs')[1].split('_')[0])
        confidence = 0.95
        if task=='prune':
            prune_rates, train_accs, val_accs, wmk_matches, target_matches_empirical, mu_natural_empirical, sigma_natural_empirical = gather_prune_stats(model_folder, confidence,pruning_type=pruning_type)
            prune_plot(prune_rates, train_accs, val_accs, wmk_matches, c, mu_natural_empirical, sigma_natural_empirical, target_matches_empirical, axs=[ax1,ax2],
                       xlabelsize=xlabelsize, xticksize=xticksize,yticksize=yticksize,ylabelsize=ylabelsize,bigtitlesize=bigtitlesize,smalltitlesize=smalltitlesize)

        elif task=='fine_tune':
            epochs, wmk_matches, mu_natural_empirical, sigma_natural_empirical, target_matches_empirical, og_trn_accs, new_trn_accs, val_accs = gather_fine_tune_stats(model_folder, confidence, ignore_zeros=True, lr=lr, lr_scale=lr_scale, seeds=seeds)#, continuation=continuation,starting_epoch=starting_epoch)
            fine_tune_plot(dataset_name, epochs, wmk_matches, c, mu_natural_empirical, sigma_natural_empirical, target_matches_empirical, og_trn_accs, new_trn_accs, val_accs, model_folder, axs=[ax1, ax2],
                           xlabelsize=xlabelsize, xticksize=xticksize,yticksize=yticksize,ylabelsize=ylabelsize,bigtitlesize=bigtitlesize,smalltitlesize=smalltitlesize)
            fig.suptitle(f'Learning Rate Scale: {lr_scale}',y=1.05)
            
        if task=='prune':
            if pruning_type=='unstructured':
                xticks = np.linspace(0,1,6)[:-1]
                xticklabels = [f'{v:.2f}' for v in xticks]
                x_lb,x_ub = -0.01,1.01
            elif pruning_type=='structured':
                xticks = np.linspace(0,0.5,6)[:-1]
                xticklabels = [f'{v:.2f}' for v in xticks]
                x_lb,x_ub = -0.01,0.501
        elif task=='fine_tune':
            xticks = xticklabels = [int(i) for i in np.linspace(0,50,6)[:-1]]
            x_lb,x_ub = -1,51

        ax1.set_title(f'Photo ({arch})' if dataset_name=='photo' else f'{dataset_name} ({arch})',fontsize=smalltitlesize)
        ax1.label_outer()
        ax2.label_outer()
        handles, labels = [],[]
        handles_0, labels_0 = ax1.get_legend_handles_labels()
        handles_1, labels_1 = ax2.get_legend_handles_labels()
        handles.extend(handles_0);  handles.extend(handles_1)
        labels.extend(labels_0);    labels.extend(labels_1)
        ax1.set_xticks(xticks)
        ax1.set_xlim(x_lb,x_ub)
        ax2.set_xticks(xticks)
        ax2.set_xticklabels(xticklabels,rotation=xrotation,fontsize=xticksize)
        ax2.set_xlim(x_lb,x_ub)

    if exclude_title==False:
        if task=='prune':
            if pruning_type=='unstructured':
                fig.suptitle(f'{pruning_type.capitalize()} Pruning',y=1.01,fontsize=bigtitlesize)
            else:
                fig.suptitle(f'Pruning',y=1.02,fontsize=bigtitlesize)
        elif task=='fine_tune':
            fig.suptitle('Fine Tuning',y=1.02,fontsize=bigtitlesize)
    task_descrip = f'{pruning_type}_pruning' if task=='prune' else 'fine_tuning'
    image_name = f'all_datasets_{arch}_{task_descrip}'
    ncols_legend=3 if task=='prune' else 2
    ylegend=-0.12 if task=='prune' else -0.15
    fig.legend(handles=handles,labels=labels,ncols=ncols_legend, loc='center', bbox_to_anchor=(0.5,ylegend),fontsize=legendsize)
    fig.subplots_adjust(wspace=0.0,hspace=0.0)

    plt.savefig(f'{projet_dir}/{image_name}.png',bbox_inches='tight',dpi=300)



# Example

dataset_architecture_dict = {'photo':'SAGE','PubMed':'SAGE','CS':'SAGE'}
arch='SAGE'
xrotation=0
xticksize=9
yticksize=9
xlabelsize=9
ylabelsize=9
bigtitlesize=13
smalltitlesize=11
legendsize=9
exclude_title=True

plot_all_datasets(dataset_architecture_dict, 4, 0.005,'fine_tune',exclude_title=exclude_title, arch='SGC',xrotation=xrotation,xticksize=xticksize,yticksize=yticksize,
                                                                                        xlabelsize=xlabelsize,ylabelsize=ylabelsize,bigtitlesize=bigtitlesize,
                                                                                        smalltitlesize=smalltitlesize,legendsize=legendsize, lr_scale=0.1)

plot_all_datasets_ablation(dataset_architecture_dict, 'num_subgraph','fine_tune',exclude_title=exclude_title, arch='SAGE',include_accs=True, xrotation=xrotation,xticksize=xticksize,yticksize=yticksize,
                                                                                        xlabelsize=xlabelsize,ylabelsize=ylabelsize,bigtitlesize=bigtitlesize,
                                                                                        smalltitlesize=smalltitlesize,legendsize=legendsize, seeds=None,project_dir=project_dir)



In [ ]:
def plot_all_fine_tune_in_grid(arch_path, subgraph_size=None):
    model_paths = [os.path.join(arch_path,f) for f in os.listdir(arch_path) if  f[0]!='.' and '.png' not in f and 'ignore' not in f]
    if subgraph_size is not None:
        model_paths = [m for m in model_paths if parse_subgraph_size(m)==subgraph_size]
    model_paths = sorted(model_paths, key=parse_num_subgraphs_from_path)

    num_cols = len(model_paths)
    fig = plt.figure(figsize=(9, 3))
    gs = gridspec.GridSpec(2, num_cols, figure=fig)

    for i, model_path in enumerate(model_paths):
        col = i
        ax1 = fig.add_subplot(gs[0,col])
        ax2 = fig.add_subplot(gs[1,col])
        dataset_name = model_path.split('training_results')[1].split('/')[1].split('/')[0]
        c = int(model_path.split('numSubgraphs')[1].split('_')[0])
        confidence = 0.95
        epochs, wmk_matches, mu_natural_empirical, sigma_natural_empirical, target_matches_empirical, og_trn_accs, new_trn_accs, val_accs = gather_fine_tune_stats(model_path, confidence, ignore_zeros=True, seeds=seeds)#, continuation=continuation,starting_epoch=starting_epoch)
        fine_tune_plot(dataset_name, epochs, wmk_matches, c, confidence, mu_natural_empirical, sigma_natural_empirical, target_matches_empirical, og_trn_accs, new_trn_accs, val_accs, model_path, axs=[ax1, ax2])
        ax1.label_outer()
        ax1.set_title(f'{c} Subgraphs',y=0.8,fontsize=8)
        ax2.label_outer()

        handles, labels = [],[]
        handles_0, labels_0 = ax1.get_legend_handles_labels()
        handles_1, labels_1 = ax2.get_legend_handles_labels()
        handles.extend(handles_0);  handles.extend(handles_1)
        labels.extend(labels_0);    labels.extend(labels_1)


    fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 0.04), ncol=4,fontsize=8)
    fig.suptitle(dataset_name.capitalize(),y=0.9,fontsize=10)
    plt.tight_layout()
    plt.subplots_adjust(wspace=0, hspace=0)
    plt.show()

def plot_all_prune_in_grid(arch_path,pruning_type='structured', subgraph_size=None):
    model_paths = [os.path.join(arch_path,f) for f in os.listdir(arch_path) if  f[0]!='.' and '.png' not in f and 'ignore' not in f]
    if subgraph_size is not None:
        model_paths = [m for m in model_paths if parse_subgraph_size(m)==subgraph_size]
    model_paths = sorted(model_paths, key=parse_num_subgraphs_from_path)
    num_cols = len(model_paths)
    fig = plt.figure(figsize=(9, 3))
    gs = gridspec.GridSpec(2, num_cols, figure=fig)

    for i, model_path in enumerate(model_paths):
        col = i
        ax1 = fig.add_subplot(gs[0,col])
        ax2 = fig.add_subplot(gs[1,col])
        dataset_name = model_path.split('training_results')[1].split('/')[1].split('/')[0]
        c = int(model_path.split('numSubgraphs')[1].split('_')[0])
        confidence = 0.95
        prune_rates, train_accs, val_accs, wmk_matches, target_matches_empirical, mu_natural_empirical, sigma_natural_empirical = gather_prune_stats(model_path, confidence,pruning_type=pruning_type)
        prune_plot(prune_rates, train_accs, val_accs, wmk_matches, c, confidence, mu_natural_empirical, sigma_natural_empirical, target_matches_empirical, model_path, use_p_instead_of_count=True, axs=[ax1,ax2])

        ax1.label_outer()
        ax1.set_title(f'{c} Subgraphs',y=0.8,fontsize=8)
        ax2.label_outer()

        handles, labels = [],[]
        handles_0, labels_0 = ax1.get_legend_handles_labels()
        handles_1, labels_1 = ax2.get_legend_handles_labels()
        handles.extend(handles_0);  handles.extend(handles_1)
        labels.extend(labels_0);    labels.extend(labels_1)


    fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 0.04), ncol=4,fontsize=8)
    fig.suptitle(dataset_name.capitalize(),y=0.9,fontsize=10)
    plt.tight_layout()
    plt.subplots_adjust(wspace=0, hspace=0)
    plt.show()

folder = '<main_directory_path>/training_results/photo/archSAGE_elu_nLayers3_hDim256_drop0.1_skipTrue'
num_cols = len([f for f in os.listdir(folder) if f[0]!='.' and '.png' not in f and 'ignore' not in f])
plot_all_fine_tune_in_grid(folder, subgraph_size=0.005)